# Battle of neighborhoods

## 1. INTRODUCTION

### PROBLEM

*It happened almost by accident.
It was Chiara and I, at the coffee shop, in front of a cappuccino, when she said to me: "I may seem crazy, but I would like to open a gym in Manhattan."
I was impressed, Chiara graduated in Law and I didn't expect her to have this idea.
"What I'm missing is to feel comfortable, I wouldn't want it to turn out to be a failure," she said.*

The question is: if my friend Chiara is looking to open a Gym in Manhattan, would I recommend that she opens it? And where?


### INTEREST

I would show this study to possible investors that could contribute in making this project real.

## 2. DATA

In this project, data was collected from:
- a json file containing New York data
- Foursquare API


In [4]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files

!pip install --user geopy

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install --user folium 
import folium # map rendering library

print('Libraries imported.')

     |████████████████████████████████| 92kB 5.5MB/s eta 0:00:011
Libraries imported.


### NY DATA COLLECTION

In [5]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [6]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [7]:
neighborhoods_data = newyork_data['features']
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [8]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

### NY DATA FORMATTING

In [9]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [10]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


In [11]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


### NY DATA EXPLORATION

#### Map of NY with neighborhoods superimposed

In [12]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

### MANHATTAN DATA FILTERING

In [13]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [14]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


### MANHATTAN DATA EXPLORATION

#### Map of Manhattan with neighborhoods superimposed

In [15]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

## Foursquare API

In [18]:
CLIENT_ID = 'ZK1EZ3ZPVHG5UIR3QBSOFE1CA31JOGQLAO1HRIKXBPZPZO24' # your Foursquare ID
CLIENT_SECRET = 'V5UPV3TIEI5PLQH5UM2NOCKYZVC1AJS44N3WJ2B5A2PPOXSQ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [20]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
neighborhood_latitude = manhattan_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = manhattan_data.loc[0, 'Longitude'] # neighborhood longitude value

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=ZK1EZ3ZPVHG5UIR3QBSOFE1CA31JOGQLAO1HRIKXBPZPZO24&client_secret=V5UPV3TIEI5PLQH5UM2NOCKYZVC1AJS44N3WJ2B5A2PPOXSQ&v=20180605&ll=40.87655077879964,-73.91065965862981&radius=500&limit=100'

### COLLECT MANHATTAN VENUES VIA THE API

In [21]:
results = requests.get(url).json()

#### Get venues and related categories

In [22]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [23]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Arturo's,Pizza Place,40.874412,-73.910271
1,Bikram Yoga,Yoga Studio,40.876844,-73.906204
2,Tibbett Diner,Diner,40.880404,-73.908937
3,Dunkin',Donut Shop,40.877136,-73.906666
4,Starbucks,Coffee Shop,40.877531,-73.905582


#### Group venues by Neighborhood

In [24]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [26]:
# run on each neighborhood
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [27]:
print(manhattan_venues.shape)
manhattan_venues.head()

(3238, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Dunkin',40.877136,-73.906666,Donut Shop
4,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop


#### Number of venues per Neighborhood

In [28]:
# how many venues per neighborhood
manhattan_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,80,80,80,80,80,80
Carnegie Hill,92,92,92,92,92,92
Central Harlem,44,44,44,44,44,44
Chelsea,100,100,100,100,100,100
Chinatown,100,100,100,100,100,100
Civic Center,100,100,100,100,100,100
Clinton,100,100,100,100,100,100
East Harlem,41,41,41,41,41,41
East Village,100,100,100,100,100,100


## DATA ANALYSIS

### Apply One Hot encoding for normalization

In [29]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Accessories Store,Acupuncturist,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,College Theater,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Smoothie Shop,Snack Place,Soba Restaurant,Soccer Field,Social 

In [137]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Accessories Store,Acupuncturist,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,College Theater,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Smoothie Shop,Snack Place,Soba Restaurant,Soccer Field,Social 

### Find the top 10 common venues per Neighborhood

In [33]:
# top 10 common venues
num_top_venues = 10

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Battery Park City----
                venue  freq
0                Park  0.08
1         Coffee Shop  0.06
2               Hotel  0.06
3       Shopping Mall  0.04
4       Memorial Site  0.04
5                 Gym  0.04
6           BBQ Joint  0.02
7      Clothing Store  0.02
8        Gourmet Shop  0.02
9  Italian Restaurant  0.02


----Carnegie Hill----
                  venue  freq
0           Coffee Shop  0.08
1                  Café  0.05
2           Yoga Studio  0.03
3  Gym / Fitness Center  0.03
4             Bookstore  0.03
5        Cosmetics Shop  0.03
6           Pizza Place  0.03
7                   Gym  0.03
8                Bakery  0.03
9   Japanese Restaurant  0.03


----Central Harlem----
                 venue  freq
0                  Bar  0.05
1          Pizza Place  0.05
2   Chinese Restaurant  0.05
3   African Restaurant  0.05
4  American Restaurant  0.05
5   Seafood Restaurant  0.05
6          Art Gallery  0.05
7    French Restaurant  0.05
8       Cosmetics Shop  0.

In [34]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [37]:
# top 10 venues
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Park,Hotel,Coffee Shop,Shopping Mall,Gym,Memorial Site,Pizza Place,Food Court,Boat or Ferry,Sandwich Place
1,Carnegie Hill,Coffee Shop,Café,Yoga Studio,Pizza Place,Wine Shop,Bakery,Gym,Japanese Restaurant,Cosmetics Shop,Bookstore
2,Central Harlem,Seafood Restaurant,Bar,African Restaurant,American Restaurant,Pizza Place,Art Gallery,French Restaurant,Chinese Restaurant,Fried Chicken Joint,Bookstore
3,Chelsea,Coffee Shop,American Restaurant,Art Gallery,Bakery,Hotel,Ice Cream Shop,Nightclub,Italian Restaurant,Speakeasy,Market
4,Chinatown,Chinese Restaurant,Cocktail Bar,American Restaurant,Spa,Hotpot Restaurant,Bakery,Optical Shop,Vietnamese Restaurant,Salon / Barbershop,Dessert Shop
5,Civic Center,Coffee Shop,Cocktail Bar,French Restaurant,Hotel,Gym / Fitness Center,Spa,American Restaurant,Sushi Restaurant,Park,Yoga Studio
6,Clinton,Theater,Italian Restaurant,Gym / Fitness Center,Coffee Shop,American Restaurant,Spa,Hotel,Sandwich Place,Gym,Mediterranean Restaurant
7,East Harlem,Mexican Restaurant,Bakery,Thai Restaurant,Pizza Place,Deli / Bodega,Latin American Restaurant,Spa,Spanish Restaurant,Sandwich Place,Steakhouse
8,East Village,Bar,Ice Cream Shop,Pizza Place,Wine Bar,Coffee Shop,Mexican Restaurant,Korean Restaurant,Speakeasy,Cocktail Bar,Ramen Restaurant
9,Financial District,Coffee Shop,Pizza Place,American Restaurant,Bar,Cocktail Bar,Gym,Gym / Fitness Center,Japanese Restaurant,Park,Wine Shop


## CLUSTERING NEIGBORHOODS

In [38]:
# set number of clusters
kclusters = 4

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 1, 1, 1, 1, 1, 1, 0, 1, 1], dtype=int32)

In [39]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,1,Sandwich Place,Coffee Shop,Gym,Yoga Studio,Shopping Mall,Steakhouse,Seafood Restaurant,Supplement Shop,Donut Shop,Miscellaneous Shop
1,Manhattan,Chinatown,40.715618,-73.994279,1,Chinese Restaurant,Cocktail Bar,American Restaurant,Spa,Hotpot Restaurant,Bakery,Optical Shop,Vietnamese Restaurant,Salon / Barbershop,Dessert Shop
2,Manhattan,Washington Heights,40.851903,-73.936900,0,Café,Bakery,Grocery Store,Chinese Restaurant,Mobile Phone Shop,Coffee Shop,Tapas Restaurant,Mexican Restaurant,Donut Shop,Latin American Restaurant
3,Manhattan,Inwood,40.867684,-73.921210,0,Mexican Restaurant,Lounge,Café,Restaurant,Pizza Place,Wine Bar,Frozen Yogurt Shop,American Restaurant,Bakery,Chinese Restaurant
4,Manhattan,Hamilton Heights,40.823604,-73.949688,0,Pizza Place,Deli / Bodega,Café,Coffee Shop,Mexican Restaurant,Yoga Studio,Sushi Restaurant,Park,Chinese Restaurant,Bakery
5,Manhattan,Manhattanville,40.816934,-73.957385,3,Coffee Shop,Deli / Bodega,Seafood Restaurant,Park,Mexican Restaurant,Italian Restaurant,Bike Trail,Bar,Grocery Store,Chinese Restaurant
6,Manhattan,Central Harlem,40.815976,-73.943211,1,Seafood Restaurant,Bar,African Restaurant,American Restaurant,Pizza Place,Art Gallery,French Restaurant,Chinese Restaurant,Fried Chicken Joint,Bookstore
7,Manhattan,East Harlem,40.792249,-73.944182,0,Mexican Restaurant,Bakery,Thai Restaurant,Pizza Place,Deli / Bodega,Latin American Restaurant,Spa,Spanish Restaurant,Sandwich Place,Steakhouse
8,Manhattan,Upper East Side,40.775639,-73.960508,1,Italian Restaurant,Coffee Shop,Gym / Fitness Center,Bakery,Yoga Studio,Hotel,Wine Shop,French Restaurant,Juice Bar,Exhibit
9,Manhattan,Yorkville,40.775930,-73.947118,1,Italian Restaurant,Gym,Coffee Shop,Bar,Deli / Bodega,Pizza Place,Japanese Restaurant,Diner,Sushi Restaurant,Mexican Restaurant


In [40]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Cluster 0

In [41]:
#cluster 0 -cafè and restaurant
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Washington Heights,Café,Bakery,Grocery Store,Chinese Restaurant,Mobile Phone Shop,Coffee Shop,Tapas Restaurant,Mexican Restaurant,Donut Shop,Latin American Restaurant
3,Inwood,Mexican Restaurant,Lounge,Café,Restaurant,Pizza Place,Wine Bar,Frozen Yogurt Shop,American Restaurant,Bakery,Chinese Restaurant
4,Hamilton Heights,Pizza Place,Deli / Bodega,Café,Coffee Shop,Mexican Restaurant,Yoga Studio,Sushi Restaurant,Park,Chinese Restaurant,Bakery
7,East Harlem,Mexican Restaurant,Bakery,Thai Restaurant,Pizza Place,Deli / Bodega,Latin American Restaurant,Spa,Spanish Restaurant,Sandwich Place,Steakhouse
36,Tudor City,Café,Park,Mexican Restaurant,Deli / Bodega,Diner,Coffee Shop,Greek Restaurant,Dog Run,Restaurant,Thai Restaurant


### Cluster 1

In [42]:
#cluster 1 - center
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Sandwich Place,Coffee Shop,Gym,Yoga Studio,Shopping Mall,Steakhouse,Seafood Restaurant,Supplement Shop,Donut Shop,Miscellaneous Shop
1,Chinatown,Chinese Restaurant,Cocktail Bar,American Restaurant,Spa,Hotpot Restaurant,Bakery,Optical Shop,Vietnamese Restaurant,Salon / Barbershop,Dessert Shop
6,Central Harlem,Seafood Restaurant,Bar,African Restaurant,American Restaurant,Pizza Place,Art Gallery,French Restaurant,Chinese Restaurant,Fried Chicken Joint,Bookstore
8,Upper East Side,Italian Restaurant,Coffee Shop,Gym / Fitness Center,Bakery,Yoga Studio,Hotel,Wine Shop,French Restaurant,Juice Bar,Exhibit
9,Yorkville,Italian Restaurant,Gym,Coffee Shop,Bar,Deli / Bodega,Pizza Place,Japanese Restaurant,Diner,Sushi Restaurant,Mexican Restaurant
10,Lenox Hill,Italian Restaurant,Coffee Shop,Pizza Place,Sushi Restaurant,Cocktail Bar,Burger Joint,Café,Gym / Fitness Center,Gym,Cycle Studio
12,Upper West Side,Italian Restaurant,Coffee Shop,Wine Bar,Bar,Bakery,Mediterranean Restaurant,Seafood Restaurant,Café,Sports Bar,Pub
13,Lincoln Square,Café,Plaza,Theater,Concert Hall,Performing Arts Venue,Gym / Fitness Center,Italian Restaurant,Cosmetics Shop,Indie Movie Theater,Wine Shop
14,Clinton,Theater,Italian Restaurant,Gym / Fitness Center,Coffee Shop,American Restaurant,Spa,Hotel,Sandwich Place,Gym,Mediterranean Restaurant
15,Midtown,Hotel,Steakhouse,Coffee Shop,Clothing Store,Sandwich Place,Sporting Goods Shop,Café,Gym,American Restaurant,Bookstore


### Cluster 2

In [43]:
#cluster 2 - harbor
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,Stuyvesant Town,Boat or Ferry,Park,Bar,Fountain,Harbor / Marina,Pet Service,Coffee Shop,Cocktail Bar,Farmers Market,Gas Station


### Cluster 3

In [44]:
#cluster 3 - parks and coffee shops
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Manhattanville,Coffee Shop,Deli / Bodega,Seafood Restaurant,Park,Mexican Restaurant,Italian Restaurant,Bike Trail,Bar,Grocery Store,Chinese Restaurant
11,Roosevelt Island,Park,Bubble Tea Shop,Gym / Fitness Center,Bridge,Metro Station,School,Supermarket,Scenic Lookout,Sandwich Place,Liquor Store
21,Tribeca,American Restaurant,Italian Restaurant,Park,Café,Coffee Shop,Wine Bar,Spa,Greek Restaurant,Basketball Court,Bakery
26,Morningside Heights,Park,Bookstore,American Restaurant,Coffee Shop,Burger Joint,Deli / Bodega,College Cafeteria,Salad Place,Seafood Restaurant,Tennis Court
28,Battery Park City,Park,Hotel,Coffee Shop,Shopping Mall,Gym,Memorial Site,Pizza Place,Food Court,Boat or Ferry,Sandwich Place
39,Hudson Yards,American Restaurant,Gym / Fitness Center,Hotel,Coffee Shop,Thai Restaurant,Italian Restaurant,Asian Restaurant,Dog Run,Gym,Park


## INTERMEDIATE RESULTS

It seems that the best and most central cluster is cluster 1. I will check in which Neighborhoods the Gym category appears in this cluster

In [45]:
cluster1 = manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

gymlocs = pd.DataFrame()
for cols in cluster1.columns:
    gymlocs= gymlocs.append(cluster1[cluster1[cols].str.contains("Gym")])
gymlocs

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Yorkville,Italian Restaurant,Gym,Coffee Shop,Bar,Deli / Bodega,Pizza Place,Japanese Restaurant,Diner,Sushi Restaurant,Mexican Restaurant
34,Sutton Place,Italian Restaurant,Gym / Fitness Center,Gym,Coffee Shop,Furniture / Home Store,Pizza Place,Yoga Studio,Beer Garden,Park,Dessert Shop
0,Marble Hill,Sandwich Place,Coffee Shop,Gym,Yoga Studio,Shopping Mall,Steakhouse,Seafood Restaurant,Supplement Shop,Donut Shop,Miscellaneous Shop
8,Upper East Side,Italian Restaurant,Coffee Shop,Gym / Fitness Center,Bakery,Yoga Studio,Hotel,Wine Shop,French Restaurant,Juice Bar,Exhibit
14,Clinton,Theater,Italian Restaurant,Gym / Fitness Center,Coffee Shop,American Restaurant,Spa,Hotel,Sandwich Place,Gym,Mediterranean Restaurant
34,Sutton Place,Italian Restaurant,Gym / Fitness Center,Gym,Coffee Shop,Furniture / Home Store,Pizza Place,Yoga Studio,Beer Garden,Park,Dessert Shop
32,Civic Center,Coffee Shop,Cocktail Bar,French Restaurant,Hotel,Gym / Fitness Center,Spa,American Restaurant,Sushi Restaurant,Park,Yoga Studio
13,Lincoln Square,Café,Plaza,Theater,Concert Hall,Performing Arts Venue,Gym / Fitness Center,Italian Restaurant,Cosmetics Shop,Indie Movie Theater,Wine Shop
16,Murray Hill,Coffee Shop,Sandwich Place,Hotel,Bar,Japanese Restaurant,Gym / Fitness Center,Restaurant,American Restaurant,Italian Restaurant,Sushi Restaurant
18,Greenwich Village,Italian Restaurant,Sushi Restaurant,Café,American Restaurant,Indian Restaurant,Gym,French Restaurant,Clothing Store,Chinese Restaurant,Gourmet Shop


## RESULTS

I will list the Neighborhoods where Gym does not appear.

In [46]:
merged = cluster1.merge(gymlocs, how='left', indicator=True)
noGymlocs = merged[merged['_merge']=='left_only'].drop(columns=['_merge'])
noGymlocs

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Chinatown,Chinese Restaurant,Cocktail Bar,American Restaurant,Spa,Hotpot Restaurant,Bakery,Optical Shop,Vietnamese Restaurant,Salon / Barbershop,Dessert Shop
2,Central Harlem,Seafood Restaurant,Bar,African Restaurant,American Restaurant,Pizza Place,Art Gallery,French Restaurant,Chinese Restaurant,Fried Chicken Joint,Bookstore
7,Upper West Side,Italian Restaurant,Coffee Shop,Wine Bar,Bar,Bakery,Mediterranean Restaurant,Seafood Restaurant,Café,Sports Bar,Pub
13,Chelsea,Coffee Shop,American Restaurant,Art Gallery,Bakery,Hotel,Ice Cream Shop,Nightclub,Italian Restaurant,Speakeasy,Market
15,East Village,Bar,Ice Cream Shop,Pizza Place,Wine Bar,Coffee Shop,Mexican Restaurant,Korean Restaurant,Speakeasy,Cocktail Bar,Ramen Restaurant
16,Lower East Side,Chinese Restaurant,Coffee Shop,Pizza Place,Cocktail Bar,Pharmacy,Art Gallery,Ramen Restaurant,Bakery,Café,Bus Stop
17,Little Italy,Café,Bakery,Italian Restaurant,Bubble Tea Shop,Cocktail Bar,Chinese Restaurant,Spa,Thai Restaurant,Sandwich Place,Hotel
18,Soho,Clothing Store,Coffee Shop,Italian Restaurant,Bakery,Boutique,Mediterranean Restaurant,Café,Salon / Barbershop,French Restaurant,Hotel
19,West Village,Italian Restaurant,New American Restaurant,Wine Bar,Jazz Club,American Restaurant,Cocktail Bar,Park,Coffee Shop,Theater,Bakery
20,Manhattan Valley,Indian Restaurant,Mexican Restaurant,Bar,Coffee Shop,Yoga Studio,Chinese Restaurant,Thai Restaurant,Pizza Place,Caribbean Restaurant,Clothing Store


## DISCUSSION

Given that in these Neighborhoods a Gym is not available, it can be favorable, as there are no competitors.<br>
To narrow the results, I believe that the presence of transport services, pharmacies, bars and restaurants and sports clothing stores can be crucial.

In [48]:
final = pd.DataFrame()
categories = ['Clothing Store', 'Restaurant', 'Bus Stop', 'Pharmacy']

for cols in noGymlocs.columns:
    for c in categories:
        final= final.append(noGymlocs[noGymlocs[cols].str.contains(c)])
final.drop_duplicates()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Soho,Clothing Store,Coffee Shop,Italian Restaurant,Bakery,Boutique,Mediterranean Restaurant,Café,Salon / Barbershop,French Restaurant,Hotel
1,Chinatown,Chinese Restaurant,Cocktail Bar,American Restaurant,Spa,Hotpot Restaurant,Bakery,Optical Shop,Vietnamese Restaurant,Salon / Barbershop,Dessert Shop
2,Central Harlem,Seafood Restaurant,Bar,African Restaurant,American Restaurant,Pizza Place,Art Gallery,French Restaurant,Chinese Restaurant,Fried Chicken Joint,Bookstore
7,Upper West Side,Italian Restaurant,Coffee Shop,Wine Bar,Bar,Bakery,Mediterranean Restaurant,Seafood Restaurant,Café,Sports Bar,Pub
16,Lower East Side,Chinese Restaurant,Coffee Shop,Pizza Place,Cocktail Bar,Pharmacy,Art Gallery,Ramen Restaurant,Bakery,Café,Bus Stop
19,West Village,Italian Restaurant,New American Restaurant,Wine Bar,Jazz Club,American Restaurant,Cocktail Bar,Park,Coffee Shop,Theater,Bakery
20,Manhattan Valley,Indian Restaurant,Mexican Restaurant,Bar,Coffee Shop,Yoga Studio,Chinese Restaurant,Thai Restaurant,Pizza Place,Caribbean Restaurant,Clothing Store
25,Noho,Italian Restaurant,Cocktail Bar,Coffee Shop,French Restaurant,Hotel,Pizza Place,Grocery Store,Yoga Studio,Art Gallery,Wine Bar
30,Turtle Bay,Italian Restaurant,Coffee Shop,Sushi Restaurant,Wine Bar,Café,Japanese Restaurant,French Restaurant,Hotel,Park,Karaoke Bar
13,Chelsea,Coffee Shop,American Restaurant,Art Gallery,Bakery,Hotel,Ice Cream Shop,Nightclub,Italian Restaurant,Speakeasy,Market


Given that restaurants are almost everywhere, I will narrow the filter.

In [49]:
final = pd.DataFrame()
categories = ['Clothing Store','Bus Stop', 'Pharmacy']

for cols in noGymlocs.columns:
    for c in categories:
        final= final.append(noGymlocs[noGymlocs[cols].str.contains(c)])
final.drop_duplicates()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Soho,Clothing Store,Coffee Shop,Italian Restaurant,Bakery,Boutique,Mediterranean Restaurant,Café,Salon / Barbershop,French Restaurant,Hotel
16,Lower East Side,Chinese Restaurant,Coffee Shop,Pizza Place,Cocktail Bar,Pharmacy,Art Gallery,Ramen Restaurant,Bakery,Café,Bus Stop
20,Manhattan Valley,Indian Restaurant,Mexican Restaurant,Bar,Coffee Shop,Yoga Studio,Chinese Restaurant,Thai Restaurant,Pizza Place,Caribbean Restaurant,Clothing Store


## CONCLUSIONS

Examining the venue categories for each Neighborhood in the final result, I would suggest:
- **Lower East Side:** has Bus Stops (good for reachability), Pharmacy (good in case of accidents), Restaurants and Cafè 
- **Soho:** has Clothing Stores, Restaurants and Cafè


## OPEN POINTS

To optimize the decision, I would analyze the prices of buildings by Neighborhood, accessing cadastral or real estate agency data.